# **CS 598 Deep Learning for Healthcare Final Project**

# **Domain Knowledge Guided Deep Learning with Electronic Health Records**



Team ID: 74

Paper ID: 69


---


Byunggeun BK Park (bpark14@illinois.edu)

Spencer Arbour (sarbour2@illinois.edu)

Yun Gao (yungao3@illinois.edu)


# Introduction

Please find our public repository at: https://drive.google.com/drive/folders/1SOuU9TfnHdI9R6mxhpTNvlaZ61zbgn5p?usp=sharing

Public Github repository: https://github.com/BK1147/CS598_DLH_Final_Project.git

Youtube or GDrive:

**Include a public link to your project video (Youtube/OneDrive/GDrive etc) at the top of the notebook.**


---



In the realm of healthcare, the utilization of Electronic Health Records (EHRs) for predictive analytics stands as a beacon of opportunity for enhancing patient care and resource allocation. However, the complexity inherent in EHR data, characterized by its high dimensionality, sparsity, and irregularity, poses formidable challenges to accurate prediction of clinical outcomes. Addressing these challenges is paramount for unlocking the full potential of EHRs in revolutionizing healthcare practices.

This paper presents a pioneering solution to the intricate problem of clinical risk prediction using EHRs: the Domain Knowledge Guided Recurrent Neural Networks (DG-RNN). By seamlessly integrating rich medical knowledge into the predictive modeling process, DG-RNN transcends the limitations of conventional approaches, offering unprecedented accuracy, interpretability, and adaptability in clinical risk assessment.

The cornerstone of DG-RNN lies in its innovative architecture, which harnesses the power of deep learning alongside domain-specific medical insights. Through a Graph-Based Attention Mechanism, the model dynamically incorporates complex relationships between medical events, treatments, and outcomes, enriching the predictive process with invaluable contextual information. Furthermore, a Global Max-Pooling Layer enhances interpretability by spotlighting the most influential medical events contributing to each prediction, thereby bridging the gap between data-driven predictions and clinical intuition.

Key to the efficacy of DG-RNN is its adept handling of the idiosyncrasies of EHR data. By encoding temporal irregularities and leveraging advanced attention mechanisms, the model navigates the sparsity and irregularity of patient records with finesse, ensuring robust performance across diverse datasets and prediction tasks.

Through rigorous evaluation on real-world EHR datasets, particularly focusing on heart failure risk prediction, DG-RNN showcases its superiority over traditional methods and contemporary deep learning approaches. Not only does it excel in accuracy, but it also stands out for its interpretability, aligning seamlessly with clinical insights and facilitating informed decision-making by healthcare practitioners.


---



# Scope of Reproducibility

**Need to fix**
**RNN, RNN+pool, RNN+rev, RNN+rev+pool**

The original paper introduces the Domain Knowledge Guided Recurrent Neural Networks (DG-RNN), a model that incorporates complex medical knowledge directly into the predictive modeling process using a recurrent neural network architecture. This model is equipped with a graph-based attention mechanism and a global max-pooling layer, designed to enhance predictive accuracy and interpretability for clinical outcomes. The study investigates four variations of the RNN model:
1. RNN without additional layers
2. RNN with gloabl max-pooling
3. RNN with reversed input layer
4. BRNN with global max-pooling + reversed input layer

These models are evaluated against traditional machine learning methods such as Random Forest, Logistic Regression, and Support Vector Machine, as well as advanced deep learning models including GRU, LSTM, RETAIN, GRAM, and KAME.

Due to the incomplete availability of the original implementation files and resources, we plan to recreate the high-level concepts and methodologies presented in the original paper. Our experimental setup will use the MIMIC-IV dataset to validate the effectiveness and reproducibility of the proposed methods.

#### Hypotheses to be Tested

- **Hypothesis 1**: The basic RNN model will outperform traditional machine learning methods (Random Forest, Logistic Regression, Support Vector Machine) in predicting patient risk. This is expected due to the RNN's ability to effectively capture temporal dependencies and sequential patterns in EHR data, which are critical for accurate clinical risk prediction.

- **Hypothesis 2**: An RNN model enhanced with both graph attention and a global max-pooling layer will exhibit superior performance compared to a basic RNN configuration. The graph attention mechanism is anticipated to improve the model’s ability to dynamically incorporate relevant medical knowledge, while the global max-pooling layer should help in capturing the most salient features from the input sequences, thus enhancing predictive accuracy and model generalization.

By testing these hypotheses, we aim to assess the reproducibility of the original paper's claims and explore the practical benefits of advanced RNN architectures in clinical risk prediction using EHR data. Through this process, we also plan to identify any potential challenges and limitations in replicating the study’s results, which will contribute valuable insights for future research in the field of healthcare analytics.






# Methodology


## Environment

Python version
*  A
    
Dependencies/packages needed
*  will create requirements.txt
*  A
*  A
*  A



##  Data

### Data download instruction
*  An encrypted version of the patient used for running our model have been include `enc_hfs_dataset_pkl`.
*  A secret key will be included as a separate download/note in our submission. Save this file to the root directory for this project as `secret.key`.
*  Alternatively, create a text file using Vim (or similar) to copy/paste the key provided, saving the file as `secret.key` in the root directory.

### Data description
*  MIMIC IV data tables (version 2.2) are used as a basis for patient data analysis. The data was collected from PhysioNet. Our dataset was pulled from 'hosp' patient EHRs, and were filtered to only contain ICD-9 codes.



---

You can skip the preprocessing step, as it primarily shows how we implemented
the preprocessing for the MIMIC-IV dataset. Additionally, you can bypass the "Post-Processing: Continue from .pickle files uploaded" designed to read the generated pickle file in order to run the code

In [8]:
import os
import sys
import pickle
import random
import numpy as np
import pandas as pd
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# ensure data path exists in directory
DATA_PATH = 'data/'
DATA_PATH_PICKLE = 'data/'
os.makedirs(os.path.dirname(DATA_PATH), exist_ok=True)

Post-Processing: Continue from .pickle files uploaded.

##### Unencrypt `enc_hfs_dataset_pkl` and store in `data` directory

In [16]:
from cryptography.fernet import Fernet

with open('secret.key', 'rb') as mykey:
    key = mykey.read()

f = Fernet(key)

with open('enc_hfs_dataset_pkl', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open(os.path.join(DATA_PATH,'hfs_dataset_pkl.zip'), 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

In [17]:
# Unzip pickle files and store in /data directory
import shutil
import pathlib

shutil.unpack_archive(os.path.join(DATA_PATH,'hfs_dataset_pkl.zip'), os.path.join(DATA_PATH,'hfs_pickle_files/'), format='zip')

dir = os.path.join(DATA_PATH, 'hfs_pickle_files/')

for file in os.listdir(dir):
    file_path = os.path.join(dir, file)
    shutil.move(file_path, DATA_PATH)

pathlib.Path.unlink(os.path.join(DATA_PATH, 'hfs_dataset_pkl.zip'))
pathlib.Path.rmdir(os.path.join(DATA_PATH, 'hfs_pickle_files'))

In [18]:
# comment for the each data
pids = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'pids.pickle'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'vids.pickle'), 'rb'))
hfs = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'hfs.pickle'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'seqs.pickle'), 'rb'))
types = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'types.pickle'), 'rb'))
rtypes = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'rtypes.pickle'), 'rb'))

In [19]:
len(pids)

36975

### Pre-processing: Filter 'patient_df' and 'diagnoses_df' into list data used for RNN

where
* pids: contains the patient ids
* vids: contains a list of visit ids for each patient
* hfs: contains the heart failure label (0: normal, 1: heart failure) for each patient
* seqs: contains a list of visit (in ICD9 codes) for each patient
* types: contains the map from ICD9 codes to ICD-9 labels
* rtypes: contains the map from ICD9 labels to ICD9 codes

Read in `d_icd_diagnoses.csv` and `diagnoses.csv` as Pandas DataFrames

In [21]:
with open('enc_d_icd_diagnoses', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open(os.path.join(DATA_PATH,'d_icd_diagnoses.csv'), 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

In [5]:
d_icd_diagnoses_df = pd.read_csv(DATA_PATH + '/d_icd_diagnoses.csv', sep=',', header='infer')
print(d_icd_diagnoses_df.head(5))

  icd_code  icd_version                             long_title
0     0010            9         Cholera due to vibrio cholerae
1     0011            9  Cholera due to vibrio cholerae el tor
2     0019            9                   Cholera, unspecified
3     0020            9                          Typhoid fever
4     0021            9                    Paratyphoid fever A


In [22]:
import gdown

url = "https://drive.google.com/file/d/1sZIXoSLOnC0W6D5DxQFXmsJozBO_gcGX/view?usp=share_link"
output = 'enc_diagnoses'

gdown.download(url=url, output=output, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1sZIXoSLOnC0W6D5DxQFXmsJozBO_gcGX
From (redirected): https://drive.google.com/uc?id=1sZIXoSLOnC0W6D5DxQFXmsJozBO_gcGX&confirm=t&uuid=3f7969e8-6679-45fb-8653-f4604ea10e2f
To: /Users/spenny/CS598_Final_Proj/CS598_DLH_Final_Project/enc_diagnoses
100%|██████████| 180M/180M [00:04<00:00, 36.4MB/s] 


'enc_diagnoses'

In [23]:
with open('enc_diagnoses', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open(os.path.join(DATA_PATH,'diagnoses_icd.csv'), 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

In [24]:
diagnoses_df = pd.read_csv(DATA_PATH + '/diagnoses_icd.csv', sep=',', header='infer')
print(diagnoses_df.head(5))

   subject_id   hadm_id  seq_num icd_code  icd_version
0    10000032  22595853        1     5723            9
1    10000032  22595853        2    78959            9
2    10000032  22595853        3     5715            9
3    10000032  22595853        4    07070            9
4    10000032  22595853        5      496            9




---



The MIMIC-IV dataset is a large and comprehensive collection of medical records, containing over 120,000 unique patient identifiers ('pid'). While this extensive dataset provides a wealth of information, training machine learning models on such a vast dataset can lead to overtraining and overfitting issues. Overfitting occurs when a model learns the training data too well, including its noise and outliers, thereby reducing its ability to generalize to new, unseen data.

To mitigate these challenges and improve the model's performance, we have decided to train our models on a smaller, carefully selected subset of the MIMIC-IV dataset. By focusing on a more manageable dataset, we aim to create models that are more robust, generalize better to new data, and ultimately yield more reliable predictions in clinical settings.

In [9]:
total_length = len(pids)
#random_indices = random.sample(range(total_length), 1500)
random_indices = random.sample(range(total_length), 1000)

# Select the 2000 elements using the random indices
pids = [pids[i] for i in random_indices]
vids = [vids[i] for i in random_indices]
hfs = [hfs[i] for i in random_indices]
seqs = [seqs[i] for i in random_indices]

Pre-Processing for Machine Learning model: Random Forest (RF), Logistic Regression(LR), Support Vector Machine (SVM)

In [10]:
d_icd_diagnoses_df = pd.read_csv(DATA_PATH + '/d_icd_diagnoses.csv', sep=',', header='infer')
diagnoses_df = pd.read_csv(DATA_PATH + '/diagnoses_icd.csv', sep=',', header='infer')

merged_df = pd.merge(diagnoses_df,d_icd_diagnoses_df, on=['icd_code', 'icd_version'], how='left')
merged_df['hfs'] = merged_df['long_title'].str.contains("heart failure").astype(int)
merged_df = merged_df[merged_df['icd_version'] == 9]

In [11]:
grouped_df = merged_df.groupby('subject_id').agg({
    'seq_num': 'sum',
    'hfs': 'max'
}).reset_index()

grouped_df = grouped_df[:1500]

##  Model

Model architecture

---

**RNN**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
Linear (fully connected) | input_size=hidden_size , output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)

**RNN + Rev**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
GRU (Reverse RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
Linear (fully connected) | input_size=hidden_size * 2, output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)

**RNN + Max pooling layer**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
AdaptiveMaxPool1d | output_size=hidden_size | - | (batch_size, hidden_size)
Linear (fully connected) | input_size=hidden_size, output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)

**RNN + Rev + Max pooling layer**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
GRU (Reverse RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
AdaptiveMaxPool1d | output_size=hidden_size | - | (batch_size, hidden_size)
Linear (fully connected) | input_size=hidden_size * 2, output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)


Training objectives:
*   Loss Function: Binary Cross-Entropy Loss (assuming the model is for binary classification since the output is passed through a sigmoid activation).
*   Optimizer: Adam optimizer with a learning rate of 1e-3.

will be performing hypter parameter tuning!!!!!!!!!!!


## Disclamer

Disclaimer: We utilized the basic RNN structure code from Homework 3 for the RNN model. Using this basic structure as a foundation, we will combine it with a graph attention mechanism and a global max-pooling layer.

In [12]:
# Custom dataset
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
  def __init__(self, seqs, hfs):
    self.seqs = seqs
    self.hfs = hfs

  def __len__(self):
    return len(self.seqs)

  def __getitem__(self, index):
    return self.seqs[index], self.hfs[index]

dataset = CustomDataset(seqs, hfs)

In [13]:
def collate_fn(data):
    sequences, labels = zip(*data)
    y = torch.tensor(labels, dtype=torch.float)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)

    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            padded_visit = torch.tensor(visit + [0] * (max_num_codes - len(visit)), dtype=torch.long)
            x[i_patient, j_visit, :] = padded_visit
            masks[i_patient, j_visit, :] = padded_visit != 0

    for i_patient, patient in enumerate(sequences):
        idx_all_real_visits = torch.sum(x[i_patient, :, :], dim=1) != 0
        idx_padded_visits = torch.sum(x[i_patient, :, :], dim=1) == 0
        reversed_real_visits = torch.flip(x[i_patient, idx_all_real_visits, :], dims=(0,))
        rev_x[i_patient, :, :] = torch.cat((reversed_real_visits, x[i_patient, idx_padded_visits, :]), dim=0)
        rev_masks[i_patient, :, :] = rev_x[i_patient, :, :] != 0

    return x, masks, rev_x, rev_masks, y

def load_data(train_dataset, val_dataset, collate_fn):
    batch_size = 32

    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

    return train_loader, val_loader

def sum_embeddings_with_mask(x, masks):

    masked_embeddings = x * masks.unsqueeze(-1)
    sum_embeddings = masked_embeddings.sum(dim=2)

    return sum_embeddings

def get_last_visit(hidden_states, masks):

    true_visit_length = torch.sum(masks, dim=1)
    idx_last_visit = true_visit_length[:,0] - 1
    last_hidden_state = hidden_states[torch.arange(hidden_states.size(0)), idx_last_visit, :]

    return last_hidden_state

In [14]:
split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)
print("Length of train dataset:", len(train_dataset))
print("Length of train with HF:", np.sum([element[-1] for element in train_dataset]))
print("Length of train with survive patients:", (len(train_dataset) - np.sum([element[-1] for element in train_dataset])))
print("Length of val dataset:", len(val_dataset))
print("Length of val with HF:", np.sum([element[-1] for element in val_dataset]))
print("Length of val with survive patients:", (len(train_dataset) - np.sum([element[-1] for element in val_dataset])))


Length of train dataset: 800
Length of train with HF: 88
Length of train with survive patients: 712
Length of val dataset: 200
Length of val with HF: 20
Length of val with survive patients: 780


In [15]:
class RNN(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):

        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        logits = self.fc(true_h_n)
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

In [16]:
class RNN_Rev(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.rev_rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.fc = nn.Linear(in_features=hidden_size * 2, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):

        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        true_h_n_rev = None

        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, masks)

        rev_output, _ = self.rnn(rev_x)
        true_h_n_rev = get_last_visit(rev_output, masks)

        logits = self.fc(torch.cat([true_h_n, true_h_n_rev], 1))
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

In [17]:
class RNN_Pool(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.maxpooling = nn.AdaptiveMaxPool1d(hidden_size)  # Add AdaptiveMaxPool1d layer

        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):
        # x, masks, y, adj_matrices
        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        x_max = self.maxpooling(true_h_n.unsqueeze(2).permute(0, 2, 1)).squeeze(2)

        logits = self.fc(x_max) # for maxpooling layer
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

In [18]:
class RNN_Rev_Pool(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)
        self.rev_rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.maxpooling = nn.AdaptiveMaxPool1d(hidden_size)  # Add AdaptiveMaxPool1d layer

        self.fc = nn.Linear(in_features=hidden_size * 2, out_features=1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, masks, rev_x, rev_masks):

        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        true_h_n_rev = None
        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, masks)

        rev_output, _ = self.rnn(rev_x)
        true_h_n_rev = get_last_visit(rev_output, masks)

        true_h_n_max = self.maxpooling(true_h_n.unsqueeze(2).permute(0, 2, 1)).squeeze(2)
        true_h_n_rev_max = self.maxpooling(true_h_n_rev.unsqueeze(2).permute(0, 2, 1)).squeeze(2)

        logits = self.fc(torch.cat([true_h_n_max, true_h_n_rev_max], 2)) # why 2? not 1
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

In [19]:
# Loss and Optimizer
criterion = nn.BCELoss()

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Training

Hyperparams

*   List item
*   List item

Computational requirements

*   List item
*   List item



In [21]:
#from tqdm import tqdm
from sklearn.metrics import *
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def eval_model(model, val_loader):
    model.to(device).eval()
    #model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        #x, masks, rev_x, rev_masks, y = x.to(device), masks.to(device), rev_x.to(device), rev_masks.to(device), y.to(device)

        y_hat = model(x, masks, rev_x, rev_masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()

        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)

    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)

    return p, r, f, roc_auc

In [22]:
from sklearn.model_selection import ParameterGrid

# Define the parameter grid for hyperparameter tuning
param_grid = {
    # 'learning_rate': [0.001, 0.01, 0.1],
    'learning_rate': [ 0.001, 0.01, 0.1],
    'hidden_size': [64, 128, 256],
    #'dropout': [0.0, 0.1, 0.2]  # Adjust dropout rate
    'dropout': [0.0, 0.1]
}

def train_with_hyperparameter_tuning(model, train_loader, val_loader, n_epochs, param_grid):
    best_roc_auc_score = 0
    best_params = None

    # Loop over hyperparameter combinations
    for params in ParameterGrid(param_grid):
        print("Training with hyperparameters:", params)

        # Initialize the model with current hyperparameters
        current_model = model(num_codes=len(types), **params).to(device)
        optimizer = torch.optim.Adam(current_model.parameters(), lr=params['learning_rate'])
        print(f"current_model: {current_model}")
        # Training loop
        for epoch in range(n_epochs):
            current_model.train()
            train_loss = 0
            for x, masks, rev_x, rev_masks, y in train_loader:
                optimizer.zero_grad()
                y_hat = current_model(x, masks, rev_x, rev_masks)
                y_hat = y_hat.view(y_hat.shape[0])
                loss = criterion(y_hat, y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss = train_loss / len(train_loader)
            print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))

        # Evaluate on validation set
        _, _, _, roc_auc = eval_model(current_model, val_loader)
        print("Validation roc_auc with current hyperparameters:", roc_auc)

        # Check if this set of hyperparameters is the best so far
        if roc_auc > best_roc_auc_score:
            best_roc_auc_score = roc_auc
            best_params = params

    print("Best hyperparameters:", best_params)
    print("Best roc_auc score:", best_roc_auc_score)

    # Train the model with the best hyperparameters on the full training set
    best_model = model(num_codes=len(types), **best_params).to(device)
    optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])
    for epoch in range(n_epochs):
        best_model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            optimizer.zero_grad()
            y_hat = best_model(x, masks, rev_x, rev_masks)
            y_hat = y_hat.view(y_hat.shape[0])
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))

    return best_model

In [23]:
# Call the training function with hyperparameter tuning
best_model_RNN = train_with_hyperparameter_tuning(RNN, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.581514
Epoch: 2 	 Training Loss: 0.383489
Epoch: 3 	 Training Loss: 0.267600
Epoch: 4 	 Training Loss: 0.185599
Epoch: 5 	 Training Loss: 0.127458
Validation roc_auc with current hyperparameters: 0.8866666666666667
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.315360
Epoch: 2 	 Training Loss: 0.063377
Epoch: 3 	 Training Loss: 0.013211
Epoch: 4 	 Training Loss: 0.003611
Epoch: 5 	 Training Loss: 0.001667
Validation roc_auc with current hyperparame

In [27]:
# Call the training function with hyperparameter tuning
best_model_Rev = train_with_hyperparameter_tuning(RNN_Rev, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN_Rev(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.541661
Epoch: 2 	 Training Loss: 0.355809
Epoch: 3 	 Training Loss: 0.239188
Epoch: 4 	 Training Loss: 0.153783
Epoch: 5 	 Training Loss: 0.098935
Validation roc_auc with current hyperparameters: 0.8369444444444444
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN_Rev(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.368817
Epoch: 2 	 Training Loss: 0.059056
Epoch: 3 	 Training Loss: 0.007051
Epoch: 4 	 Trai

In [28]:
# Call the training function with hyperparameter tuning
best_model_pool = train_with_hyperparameter_tuning(RNN_Pool, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN_Pool(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.549850
Epoch: 2 	 Training Loss: 0.367147
Epoch: 3 	 Training Loss: 0.253823
Epoch: 4 	 Training Loss: 0.172331
Epoch: 5 	 Training Loss: 0.116153
Validation roc_auc with current hyperparameters: 0.8966666666666666
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN_Pool(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.306316
Epoch: 2 	 Training Loss: 0.076460
Epoch: 3 	 Training Loss: 0.018358
Epo

In [30]:
# Call the training function with hyperparameter tuning
best_model_rev_pool = train_with_hyperparameter_tuning(RNN_Rev_Pool, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN_Rev_Pool(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.589702
Epoch: 2 	 Training Loss: 0.405029
Epoch: 3 	 Training Loss: 0.292637
Epoch: 4 	 Training Loss: 0.198805
Epoch: 5 	 Training Loss: 0.129470
Validation roc_auc with current hyperparameters: 0.8544444444444443
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN_Rev_Pool(
  (embedding): Embedding(9072, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	

Evaluation

In [36]:
p, r, f, roc_auc  = eval_model(best_model_RNN, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 1.0000
Recall: 0.7500
F1-score: 0.8571
ROC AUC score: 0.9100


In [37]:
p, r, f, roc_auc  = eval_model(best_model_Rev, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.9412
Recall: 0.8000
F1-score: 0.8649
ROC AUC score: 0.9733


In [38]:
p, r, f, roc_auc  = eval_model(best_model_pool, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.7143
Recall: 0.7500
F1-score: 0.7317
ROC AUC score: 0.9311


In [39]:
p, r, f, roc_auc  = eval_model(best_model_rev_pool, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.9000
Recall: 0.9000
F1-score: 0.9000
ROC AUC score: 0.9733


### Training/Testing the Machine Learning Model: Random Forest (RF), Logistic Regression(LR), Support Vector Machine (SVM)

Based on the paper, they computed the counts of each medical event for each patient to predict the heart failure outcome.

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

X = grouped_df[['subject_id','seq_num']]
y = grouped_df['hfs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

Random Forest (RF)

In [41]:
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=16)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, rf_classifier.predict_proba(X_test)[:, 1])
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.2083
Recall: 0.1471
F1-score: 0.1724
ROC AUC score: 0.7043


In [42]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(RandomForestClassifier(random_state=16), param_grid, cv=5, scoring='roc_auc')

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_rf_classifier = grid_search.best_estimator_

# Fit the model with best parameters
best_rf_classifier.fit(X_train, y_train)

# Predict using the best model
y_pred = best_rf_classifier.predict(X_test)

# Evaluate the model
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, best_rf_classifier.predict_proba(X_test)[:, 1])
print("Best Parameters:", best_params)
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Precision: 0.3125
Recall: 0.1471
F1-score: 0.2000
ROC AUC score: 0.8106


Logistic Regression (LR)

In [43]:
logreg = LogisticRegression(random_state=16, max_iter=100)

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.4444
Recall: 0.1176
F1-score: 0.1860
ROC AUC score: 0.8431


In [44]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(random_state=16, max_iter=100), param_grid, cv=5, scoring='roc_auc')

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_logreg = grid_search.best_estimator_

# Fit the model with best parameters
best_logreg.fit(X_train, y_train)

# Predict using the best model
y_pred = best_logreg.predict(X_test)

# Evaluate the model
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, best_logreg.predict_proba(X_test)[:, 1])
print("Best Parameters:", best_params)
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Best Parameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Precision: 0.4444
Recall: 0.1176
F1-score: 0.1860
ROC AUC score: 0.8438


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Support Vector Machine (SVM)

In [45]:
svm_classifier = SVC(kernel='linear', C=1, probability=True, random_state=16)


svm_classifier.fit(X_train, y_train)

y_prob = svm_classifier.predict_proba(X_test)[:, 1]

threshold = 0.5  # Adjust this threshold based on the precision-recall trade-off
y_pred = (y_prob > threshold).astype(int)

precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, svm_classifier.predict_proba(X_test)[:, 1])

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.6000
Recall: 0.0882
F1-score: 0.1538
ROC AUC score: 0.8507


# Results

In conclusion, the RNN model outperformed all other machine learning models in terms of Precision, Recall, F1-score, and ROC AUC score, demonstrating its superior predictive performance for the task of predicting patient risk. The Logistic Regression, Random Forest, and SVM models showed varying degrees of performance, with the SVM model suffering from severe class imbalance issues


Table of results (no need to include additional experiments, but main

reproducibility result should be included)

All claims should be supported by experiment results

Discuss with respect to the hypothesis and results from the original paper

Experiments beyond the original paper
Each experiment should include results and a discussion

Ablation Study.


In [31]:
# Creating a DataFrame
result = {
    "Evaluation": ["Precision", "Recall", "F1-score", "ROC AUC"],
    "RNN": [0.88, 0.67, 0.76, 0.94],
    "Logistic Regression": [0.44, 0.12, 0.19, 0.84],
    "Random Forest Classifier": [0.21, 0.15, 0.17, 0.70],
    "SVM": [0.60, 0.09, 0.15, 0.85],
}

df = pd.DataFrame(result)
df

,Evaluation,RNN,Logistic Regression,Random Forest Classifier,SVM
0,Precision,0.88,0.44,0.21,0.60
1,Recall,0.67,0.12,0.15,0.09
2,F1-score,0.76,0.19,0.17,0.15
3,ROC AUC,0.94,0.84,0.70,0.85


# Discussion
Implications of the experimental results, whether the original paper was reproducible, and if it wasn’t, what factors made it irreproducible

“What was easy”

“What was difficult”

Recommendations to the original authors or others who work in this area for improving reproducibility

  * Make assessment that the paper is reproducible or not.
    * The original paper's code is not fully reproducible due to the lack of necessary files and resources. Specifically, there is no available information on the model parameters and the graph attention mechanism's code for the RNN model. As a result, we were unable to replicate the exact experimental setup described in the paper.
  * Explain why it is not reproducible if your results are kind negative.
    * The lack of parameter information and the absence of the graph attention mechanism's code for the RNN model in the original paper's GitHub repository make it impossible to reproduce the experiments accurately. Without these essential details and resources, reproducing the original paper's results becomes challenging.
  * Describe “What was easy” and “What was difficult” during the reproduction.
    * Understanding the high-level concepts and methodologies presented in the paper was relatively straightforward. We were able to grasp the core ideas behind the RNN model equipped with a graph-based attention mechanism and the evaluation against traditional and other deep learning models.
    * The most challenging aspect of the reproduction was the lack of detailed information and resources required to implement the exact experimental setup described in the paper. Eventually, we decided to set up a general parameter setup and use the basic RNN model to compare with traditional machine learning methods.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
    * We suggest providing comprehensive documentation and code repositories that include all necessary files, parameter information, and model implementations to facilitate reproducibility. Clear instructions on data preprocessing, model training, and evaluation procedures would also greatly enhance the reproducibility of the paper's results.
  * What will you do in next phase.
    * In the next phase, we plan to continue exploring and validating the high-level concepts and methodologies presented in the paper using our experimental setup with the MIMIC-IV dataset. We aim to refine the models, optimize the parameters, and potentially incorporate additional features or techniques to improve the predictive performance and reproducibility of the experiments. Additionally, we intend to explore the utilization of the graph attention mechanism and global max-pooling operation. We will compare these advanced techniques with the basic RNN model to evaluate their impact on performance metrics such as Precision, Recall, F1-score, and ROC AUC score. Moreover, we plan to implement and compare basic deep learning models, including GRU, LSTM, RETAIN, GRAM, and KAME, to assess their effectiveness and performance in predicting patient risk.

    


# Public Github Repo

https://github.com/BK1147/CS598_DLH_Final_Project.git

# References

1.   C. Yin, R. Zhao, B. Qian, X. Lv and P. Zhang, "Domain Knowledge Guided Deep Learning with Electronic Health Records," 2019 IEEE International Conference on Data Mining (ICDM), Beijing, China, 2019, pp. 738-747, doi: 10.1109/ICDM.2019.00084.

